# 多目标损失融合
:label:`multi_loss_optim`

多目标往往伴随着多个损失的联合优化，这类优化方法更多的考虑的是在模型结构已经确定的条件下，结合任务的特点对模型进行训练和参数优化。简单的多目标Loss优化，是通过手工结合业务经验设定不同损失的权重，将多个损失加权为一个进行优化，如下所示：

$$Loss_{total} = \sum_i w_i L_i$$

其中，$L_i$和$w_i$分别表示第i个任务的损失及对应的权重。

在多目标建模中，当模型结构确定后，损失函数融合策略成为决定模型性能的关键因素。传统的手工加权方法存在三个本质性缺陷：

- 量级失衡问题：不同任务的损失值量级差异显著（如CTR损失通常在0.1-0.5，CVR损失可达2.0+），导致大损失主导优化方向

- 收敛异步问题：稀疏任务收敛慢，密集任务收敛快，造成过拟合与欠拟合并存

- 梯度冲突问题：任务梯度方向不一致时产生抵消效应（如CTR与CTR任务梯度夹角>90°）

下面系统解析三大主流优化方法，包含理论框架、实现机制与工程实践。

## Uncertainty Weight：基于不确定性的自适应加权

基于不确定性加权损失（Uncertainty Weighted Loss, UWL）。UWL的核心思想是根据任务的不确定性动态调整权重，具体来说，任务的损失越大，分配的权重越小。论文指出，在任务训练过程中存在两种不确定性：一种是认知不确定性（epistemic uncertainty），源于数据的缺乏；另一种是偶然不确定性（aleatoric uncertainty），源于数据本身或任务本身的特性。
在UWL中，任务的损失函数可以表示为：

$$
Loss = \approx \frac{1}{2\sigma_1^2} \mathcal{L}_1(\mathbf{W}) + \frac{1}{\sigma_2^2} \mathcal{L}_2(\mathbf{W}) + \log \sigma_1 + \log \sigma_2
$$

其中，$\sigma$表示的是任务的不确定性(uncertainty),是可学习的参数。从公式可以看出，当loss较大且$\sigma$较小时，$\frac{1}{2\sigma^2} \mathcal{L}(\mathbf{W})$会很大，损失函数在优化的时候就会将其往小了优化。可以直观的理解为，模型不会让任务往不确定性较大的方向大幅更新参数。


## GradNorm：梯度标准化方法

在多任务优化的过程中，不同的任务loss的量级是不一样的，这样带来的问题就是loss大的任务梯度更新的幅度也会更大，进而导致模型在学习的过程中被loss大的任务主导带偏整个模型。此外，不同的任务由于数据分布的原因，loss的收敛速度也是不同的。为了同时考虑loss的量级和训练的速度。GradNorm在模型优化过程中除了正常的任务loss外，还引入了一个gradient loss，该loss通过梯度下降的方式来更新不同任务的loss权重。并且这两个loss是单独优化的，而不是简单的相加得到一个loss去综合优化。

在介绍gradient loss之前，我们先来看一下如何定义梯度的量级和loss的学习速度。

$$
G_{W}^{(i)}(t) \, = \, \|\nabla_{W} w_{i}(t) L_{i}(t)\|_{2}
$$

$$
\overline{G}_{W}(t) \, = \, E_{\text{task}}[G_{W}^{(i)}(t)]
$$

其中$W$是所有任务loss对多个任务最后一层共享参数，$G_{W}^{(i)}(t)$表示任务$i$加权后的Loss，对共享参数$W$的梯度，该值较大时表示loss $i$当前的梯度量级较大，$\overline{G}_{W}(t)$表示所有任务对共享参数梯度的均值。


$$
\tilde{L}_i(\tilde{t}) = L_i(t) / L_i(0)
$$

$$r_i(t) = \frac{\tilde{L}_i(t)}{E_{\text{task}}[\tilde{L}_i(t)]}$$

$L_i(t)$表示的是训练的第t时刻，任务$i$的Loss值，所以$\tilde{L}_i(\tilde{t})$表示的是任务$i$在第t时刻的相对第0时刻的损失比率，该值如果越小的话则代表该任务loss收敛的比较快，训练速度较大。$r_i(t)$则是在$L_i(t)$的基础上做了一次归一化，让所有任务之间的速率相对可以比较，同样也是值越小表示任务的训练速度越快。

最终的梯度损失函数定义为如下表达式：
$$
L_{\text{grad}}(t; w_i(t)) = \sum_i \left| G_W^{(i)}(t) - \overline{G}_W(t) \times [r_i(t)]^\alpha \right|_1
$$

梯度损失函数综合了上述定义的梯度量级和学习速度，直观理解就是当某个loss的梯度非常大时，该loss的值也会较大进而会将该loss的权重降的更小，避免了梯度大的loss主导了模型的学习。同理当某个任务学习的速度较快时，即$r_i(t)$较小，梯度loss也会变得更大，进而使得该loss的权重会变得更小，阻止某个任务过快的收敛。

## Pareto Optimization：帕累托优化框架

在多任务学习中，当不同任务的梯度方向存在根本性冲突时（即优化任务A必然损害任务B），我们面临帕累托边界优化问题。传统加权平均方法在此场景下失效，需要专门的优化框架寻找帕累托最优解集：

$$
\min_{\theta} \mathbf{L}(\theta) = \min_{\theta} (\mathcal{L}_1(\theta), \mathcal{L}_2(\theta), ..., \mathcal{L}_T(\theta))
$$

其中帕累托最优解定义为：不存在其他解能在不损害至少一个任务的情况下改进任一任务。


**帕累托最优损失融合核心思想：**

将多目标损失合并为加权和，并利用 KKT 条件动态调整权重，使优化方向指向帕累托前沿：

$$
\mathcal{L}(\theta) = \sum_{i=1}^{K} w_i \mathcal{L}_i (\theta)
$$

其中 $w_i$ 为可学习的权重，满足 $\sum w_i = 1$ 且 $w_i \geq c_i$（$c_i$ 为权重下限）。

**优化步骤（分两步迭代）：**

1. 固定权重，更新模型参数 $\theta$：通过梯度下降最小化加权损失 $\mathcal{L}(\theta)$，即常规的模型训练步骤。

2. 固定模型，更新权重 $w_i$
    - 目标：求解权重 $w_i$，使加权梯度的二范数最小化（满足 KKT 条件）：
    $$
    \min _{w}\left\|\sum_{i=1}^{K} w_{i} \nabla_{\theta} \mathcal{L}_{i}(\theta)\right\|_{2}^{2}
    $$
    - 约束条件： $\sum w_i = 1$，$w_i \geq c_i$。
    - 松弛与投影：
        - 引入变量 $\tilde{w}_i = w_i - c_i$，将不等式约束转化为非负约束。
        - 先忽略 $\tilde{w}_i \geq 0$，求解带等式约束的二次规划问题。
        - 对解 $\tilde{w}^*$ 进行投影，确保其非负性（投影问题可通过闭式解快速求解）：
        $$
        \min _{\tilde{w}}\|\tilde{w}-\tilde{w}^*\|_2^2 \quad \text{s.t.} \quad \sum \tilde{w}_i=1-\sum c_i, \tilde{w}_i \geq 0
        $$


PE-LTR的核心贡献在于将多目标优化的帕累托条件转化为权重学习的二次规划问题，通过交替更新模型参数与损失权重，引导模型收敛至帕累托前沿。